In [47]:
import pandas as pd
import os
from datetime import datetime

In [48]:
file_list = os.listdir()
file_list = [name for name in file_list if name.startswith("olist")]
file_list

['olist_orders_dataset.csv',
 'olist_customers_dataset.csv',
 'olist_order_items_dataset.csv']

In [49]:
files = {}
for file in file_list:
    files['{}'.format(file).split('.')[0]] = pd.read_csv('{}'.format(file))
olist_orders_dataset = files['olist_orders_dataset']
olist_customers_dataset = files['olist_customers_dataset']
olist_order_items_dataset = files['olist_order_items_dataset']

In [50]:
one_order_customers = (
    olist_orders_dataset.merge(olist_customers_dataset, how='inner', on='customer_id')
        .groupby('customer_unique_id')
        .agg({'order_id': 'count'})
        .reset_index()
        .query("order_id == 1")['customer_unique_id']
        .nunique()
)
one_order_customers
# first task - 93099

93099

In [51]:
not_customer_delivered = olist_orders_dataset[olist_orders_dataset['order_delivered_customer_date'].isna()]

In [52]:
not_customer_delivered['month_purchase'] = pd.to_datetime(not_customer_delivered['order_purchase_timestamp']).dt.to_period('M')
not_customer_delivered

/tmp/ipykernel_130894/708013749.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  not_customer_delivered['month_purchase'] = pd.to_datetime(not_customer_delivered['order_purchase_timestamp']).dt.to_period('M')


,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,month_purchase
6,136cce7faa42fdb2cefd53fdc79a6098,ed0271e0b7da060a393796590e7b737a,invoiced,2017-04-11 12:22:08,2017-04-13 13:25:17,NaN,NaN,2017-05-09 00:00:00,2017-04
44,ee64d42b8cf066f35eac1cf57de1aa85,caded193e8e47b8362864762a83db3c5,shipped,2018-06-04 16:44:48,2018-06-05 04:31:18,2018-06-05 14:32:00,NaN,2018-06-28 00:00:00,2018-06
103,0760a852e4e9d89eb77bf631eaaf1c84,d2a79636084590b7465af8ab374a8cf5,invoiced,2018-08-03 17:44:42,2018-08-07 06:15:14,NaN,NaN,2018-08-21 00:00:00,2018-08
128,15bed8e2fec7fdbadb186b57c46c92f2,f3f0e613e0bdb9c7cee75504f0f90679,processing,2017-09-03 14:22:03,2017-09-03 14:30:09,NaN,NaN,2017-10-03 00:00:00,2017-09
154,6942b8da583c2f9957e990d028607019,52006a9383bf149a4fb24226b173106f,shipped,2018-01-10 11:33:07,2018-01-11 02:32:30,2018-01-11 19:39:23,NaN,2018-02-07 00:00:00,2018-01
...,...,...,...,...,...,...,...,...,...
99283,3a3cddda5a7c27851bd96c3313412840,0b0d6095c5555fe083844281f6b093bb,canceled,2018-08-31 16:13:44,NaN,NaN,NaN,2018-10-01 00:00:00,2018-08
99313,e9e64a17afa9653aacf2616d94c005b8,b4cd0522e632e481f8eaf766a2646e86,processing,2018-01-05 23:07:24,2018-01-09 07:18:05,NaN,NaN,2018-02-06 00:00:00,2018-01
99347,a89abace0dcc01eeb267a9660b5ac126,2f0524a7b1b3845a1a57fcf3910c4333,canceled,2018-09-06 18:45:47,NaN,NaN,NaN,2018-09-27 00:00:00,2018-09
99348,a69ba794cc7deb415c3e15a0a3877e69,726f0894b5becdf952ea537d5266e543,unavailable,2017-08-23 16:28:04,2017-08-28 15:44:47,NaN,NaN,2017-09-15 00:00:00,2017-08


In [64]:
month_not_delivered = not_customer_delivered.groupby(['month_purchase', 'order_status']).agg({'order_id': 'count'}).query("order_status != 'delivered'").reset_index()
month_not_delivered.groupby('month_purchase').agg({'order_id': 'sum'}).agg({'order_id': 'mean'})
# second task answer

order_id    123.208333
dtype: float64

In [65]:
month_not_delivered.groupby('order_status').agg({'order_id': 'mean'}).round(0)
# created - order was created by a customer but it was not approved by the company
# approved - the order was approved by the company but an invoice was not created or there was some kind of a problem with it
# invoiced - an invoce was went to the customer but was not paid
# processing - when the order was paid for in an invoice but the order could not be collected
# shipped - the order was shipped by the company but never delivered by the shipping service
# cancelled - the order was cancelled a client
# unavailable - not sure???
# the short answer - during the whole process of order creation and delivery there appeared problems specific to that link in the process

,order_id
order_status,
approved,1.0
canceled,26.0
created,2.0
invoiced,15.0
processing,16.0
shipped,48.0
unavailable,29.0
